In [12]:
import numpy as np
import pandas as pd
import time
import os 

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold, cross_validate

#优化器
from bayes_opt import BayesianOptimization

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

import optuna

In [13]:
data = pd.read_csv("E:/机器学习_冲刺班/Lesson 09.随机森林模型/datasets/House Price/train_encode.csv")

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [14]:
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    
    #定义评估器
    #需要调整的超参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    #默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RFR(n_estimators = int(n_estimators)
              ,max_depth = int(max_depth)
              ,max_features = int(max_features)
              ,min_impurity_decrease = min_impurity_decrease
              ,random_state=1412
              ,verbose=False #可自行决定是否开启森林建树的verbose
              ,n_jobs=-1)
    
    #定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    #注意，交叉验证需要使用数据，但我们不能让数据X,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise'
                                     #如果交叉验证中的算法执行报错，则告诉我们错误的理由
                                    )
    
    #交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    #目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

In [15]:
param_grid_simple = {'n_estimators': (80,100)
                     , 'max_depth':(10,25)
                     , "max_features": (10,20)
                     , "min_impurity_decrease":(0,1)
                    }

In [16]:
def param_bayes_opt(init_points,n_iter):
    
    #定义优化器，先实例化优化器
    opt = BayesianOptimization(bayesopt_objective #需要优化的目标函数
                               ,param_grid_simple #备选参数空间
                               ,random_state=1412 #随机数种子，虽然无法控制住
                              )
    
    #使用优化器，记住bayes_opt只支持最大化
    opt.maximize(init_points = init_points #抽取多少个初始观测值
                 , n_iter=n_iter #一共观测/迭代多少次
                )
    
    #优化完成，取出最佳参数与最佳分数
    params_best = opt.max["params"]
    score_best = opt.max["target"]
    
    #打印最佳参数与最佳分数
    print("\n","\n","best params: ", params_best,
          "\n","\n","best cvscore: ", score_best)
    
    #返回最佳参数与最佳分数
    return params_best, score_best

In [17]:
def bayes_opt_validation(params_best):
    
    reg = RFR(n_estimators = int(params_best["n_estimators"]) 
              ,max_depth = int(params_best["max_depth"])
              ,max_features = int(params_best["max_features"])
              ,min_impurity_decrease = params_best["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1)

    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                    )
    return np.mean(validation_loss["test_score"])

In [18]:
start = time.time()
params_best, score_best = param_bayes_opt(20,280) #初始看20个观测值，后面迭代280次
print('It takes %s minutes' % ((time.time() - start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
| 1         | -2.934e+0 | 23.2      | 17.52     | 0.06379   | 88.79     |
| 2         | -2.933e+0 | 14.8      | 17.61     | 0.9214    | 97.58     |
| 3         | -2.94e+04 | 15.86     | 15.56     | 0.2661    | 87.98     |
| 4         | -2.895e+0 | 14.05     | 16.84     | 0.06744   | 89.72     |
| 5         | -2.982e+0 | 18.71     | 19.17     | 0.9315    | 83.7      |
| 6         | -2.938e+0 | 17.7      | 19.58     | 0.7127    | 89.18     |
| 7         | -2.925e+0 | 14.21     | 12.62     | 0.3381    | 91.51     |
| 8         | -2.906e+0 | 23.23     | 10.89     | 0.6078    | 95.06     |
| 9         | -2.969e+0 | 14.89     | 14.0      | 0.9487    | 80.16     |
| 10        | -2.944e+0 | 11.52     | 12.58     | 0.03276   | 92.56     |
| 11        | -2.961e+0 | 13.14     | 13.31     | 0.2563    | 98.24     |
| 12        | -2.952e+0 | 17.94     | 

| 67        | -2.891e+0 | 14.55     | 16.73     | 0.3588    | 88.84     |
| 68        | -2.942e+0 | 14.54     | 17.6      | 0.06964   | 87.39     |
| 69        | -2.891e+0 | 14.74     | 16.2      | 0.9127    | 88.97     |
| 70        | -2.891e+0 | 14.08     | 16.45     | 0.7028    | 88.92     |
| 71        | -2.934e+0 | 19.28     | 17.77     | 0.1678    | 92.86     |
| 72        | -2.891e+0 | 14.92     | 16.61     | 0.6595    | 88.53     |
| 73        | -2.905e+0 | 18.17     | 17.02     | 0.0       | 95.05     |
| 74        | -2.907e+0 | 17.9      | 17.7      | 0.4353    | 97.62     |
| 75        | -2.916e+0 | 17.92     | 16.79     | 0.2871    | 98.03     |
| 76        | -2.969e+0 | 17.26     | 18.34     | 0.2133    | 98.29     |
| 77        | -2.902e+0 | 18.24     | 17.03     | 0.5449    | 98.95     |
| 78        | -2.902e+0 | 18.27     | 17.47     | 0.9998    | 98.32     |
| 79        | -2.938e+0 | 19.11     | 17.47     | 0.2247    | 98.61     |
| 80        | -2.913e+0 | 17.53     | 

| 135       | -2.869e+0 | 10.25     | 16.57     | 0.1563    | 89.93     |
| 136       | -2.969e+0 | 20.93     | 19.89     | 0.0004996 | 91.1      |
| 137       | -2.869e+0 | 10.29     | 16.49     | 0.3564    | 89.61     |
| 138       | -2.869e+0 | 10.03     | 16.55     | 0.5935    | 89.94     |
| 139       | -2.869e+0 | 10.14     | 16.03     | 0.2995    | 89.78     |
| 140       | -2.951e+0 | 22.91     | 19.65     | 0.8711    | 97.25     |
| 141       | -2.879e+0 | 11.0      | 16.27     | 0.5273    | 86.79     |
| 142       | -2.921e+0 | 17.9      | 16.99     | 0.8974    | 83.97     |
| 143       | -2.879e+0 | 10.75     | 16.52     | 0.9562    | 86.77     |
| 144       | -2.87e+04 | 10.52     | 16.35     | 0.4036    | 90.03     |
| 145       | -2.869e+0 | 10.61     | 16.04     | 0.5285    | 89.7      |
| 146       | -2.869e+0 | 10.98     | 16.23     | 0.0562    | 89.54     |
| 147       | -2.879e+0 | 10.08     | 16.57     | 0.2145    | 85.69     |
| 148       | -2.869e+0 | 10.99     | 

| 203       | -2.868e+0 | 10.0      | 16.4      | 0.4624    | 91.07     |
| 204       | -2.874e+0 | 10.16     | 16.43     | 0.5244    | 87.26     |
| 205       | -2.945e+0 | 11.76     | 12.2      | 0.7708    | 85.51     |
| 206       | -2.87e+04 | 10.4      | 16.59     | 0.4192    | 90.86     |
| 207       | -2.868e+0 | 10.17     | 16.5      | 0.0       | 91.25     |
| 208       | -2.869e+0 | 10.83     | 16.42     | 0.6176    | 89.39     |
| 209       | -2.953e+0 | 23.49     | 15.7      | 0.04256   | 81.9      |
| 210       | -2.874e+0 | 10.11     | 16.54     | 0.1168    | 87.72     |
| 211       | -2.963e+0 | 10.0      | 15.92     | 0.07981   | 91.41     |
| 212       | -2.981e+0 | 18.49     | 11.06     | 0.05381   | 91.37     |
| 213       | -2.879e+0 | 10.09     | 16.99     | 0.7705    | 86.85     |
| 214       | -2.868e+0 | 10.54     | 16.96     | 0.2391    | 91.48     |
| 215       | -2.947e+0 | 18.64     | 12.77     | 0.9633    | 80.72     |
| 216       | -2.868e+0 | 10.24     | 

| 271       | -2.865e+0 | 10.24     | 16.58     | 0.044     | 92.55     |
| 272       | -2.868e+0 | 10.6      | 16.46     | 0.7428    | 94.61     |
| 273       | -2.938e+0 | 24.74     | 18.81     | 0.06543   | 85.4      |
| 274       | -2.965e+0 | 10.38     | 15.83     | 0.9038    | 94.71     |
| 275       | -2.957e+0 | 22.47     | 19.82     | 0.6159    | 86.79     |
| 276       | -2.937e+0 | 11.06     | 16.66     | 0.4028    | 95.09     |
| 277       | -2.946e+0 | 13.4      | 16.5      | 0.7104    | 80.12     |
| 278       | -2.939e+0 | 15.77     | 12.88     | 0.7749    | 90.44     |
| 279       | -2.879e+0 | 10.06     | 16.82     | 0.195     | 86.88     |
| 280       | -2.932e+0 | 24.57     | 12.36     | 0.5219    | 89.39     |
| 281       | -2.865e+0 | 10.86     | 16.85     | 0.8971    | 92.87     |
| 282       | -2.959e+0 | 13.77     | 14.47     | 0.9724    | 91.32     |
| 283       | -2.948e+0 | 10.5      | 17.25     | 0.06078   | 89.3      |
| 284       | -2.965e+0 | 10.58     | 

In [21]:
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

In [22]:
def hyperopt_objective(params):
    
    reg = RFR(n_estimators = int(params["n_estimators"])
              ,max_depth = int(params["max_depth"])
              ,max_features = int(params["max_features"])
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1)
    
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise'
                                    )
    
    return np.mean(abs(validation_loss["test_score"]))

In [23]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",80,100,1)
                     , 'max_depth': hp.quniform("max_depth",10,25,1)
                     , "max_features": hp.quniform("max_features",10,20,1)
                     , "min_impurity_decrease":hp.quniform("min_impurity_decrease",0,5,1)
                    }

In [24]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(100)
    
    #定义代理模型
    #algo = partial(tpe.suggest, n_startup_jobs=20, n_EI_candidates=50)
    params_best = fmin(hyperopt_objective #目标函数
                       , space = param_grid_simple #参数空间
                       , algo = tpe.suggest #代理模型你要哪个呢？
                       #, algo = algo
                       , max_evals = max_evals #允许的迭代次数
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [25]:
def hyperopt_validation(params):    
    reg = RFR(n_estimators = int(params["n_estimators"])
              ,max_depth = int(params["max_depth"])
              ,max_features = int(params["max_features"])
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1
             )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                    )
    return np.mean(abs(validation_loss["test_score"]))

In [26]:
params_best, trials = param_hyperopt(100)

100%|██████████| 100/100 [00:55<00:00,  1.81trial/s, best loss: 28649.507326468643]

 
 best params:  {'max_depth': 10.0, 'max_features': 16.0, 'min_impurity_decrease': 2.0, 'n_estimators': 92.0} 



In [27]:
params_best, trials = param_hyperopt(300) 

 75%|███████▌  | 226/300 [02:13<00:43,  1.70trial/s, best loss: 28649.507958829832]

 
 best params:  {'max_depth': 10.0, 'max_features': 16.0, 'min_impurity_decrease': 3.0, 'n_estimators': 92.0} 



In [28]:
hyperopt_validation(params_best)

28649.507958829832

In [29]:
trials.trials[0]

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': 29484.585160647344, 'status': 'ok'},
 'misc': {'tid': 0,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'max_depth': [0],
   'max_features': [0],
   'min_impurity_decrease': [0],
   'n_estimators': [0]},
  'vals': {'max_depth': [24.0],
   'max_features': [15.0],
   'min_impurity_decrease': [4.0],
   'n_estimators': [82.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2023, 3, 5, 1, 39, 6, 232000),
 'refresh_time': datetime.datetime(2023, 3, 5, 1, 39, 10, 936000)}

In [30]:
import optuna

In [31]:
def optuna_objective(trial):
    
    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",80,100,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",10,25,1)
    max_features = trial.suggest_int("max_features",10,20,1)
    #max_features = trial.suggest_categorical("max_features",["log2","sqrt","auto"]) #字符型
    min_impurity_decrease = trial.suggest_int("min_impurity_decrease",0,5,1)
    #min_impurity_decrease = trial.suggest_float("min_impurity_decrease",0,5,log=False) #浮点型
    
    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    reg = RFR(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_impurity_decrease = min_impurity_decrease
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1
             )
    
    #交叉验证过程，输出负均方根误差(-RMSE)
    #optuna同时支持最大化和最小化，因此如果输出-RMSE，则选择最大化
    #如果选择输出RMSE，则选择最小化
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv #交叉验证模式
                                     ,verbose=False #是否打印进程
                                     ,n_jobs=-1 #线程数
                                     ,error_score='raise'
                                    )
    #最终输出RMSE
    return np.mean(abs(validation_loss["test_score"]))

In [32]:
def optimizer_optuna(n_trials, algo):
    
    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 10, n_ei_candidates = 24)
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':10, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )
    
    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                , direction="minimize" #优化的方向，可以填写minimize或maximize
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   , n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   , show_progress_bar=True #要不要展示进度条呀？
                  )
    
    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")
    
    return study.best_trial.params, study.best_trial.values

In [33]:
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')

In [34]:
best_params, best_score = optimizer_optuna(10,"GP")

[I 2023-03-04 20:41:48,927] A new study created in memory with name: no-name-d2c70cd7-5b5e-4d2a-b10c-c00c2f3272b8
C:\Users\WenBi\anaconda3\envs\24723_Pytorch\lib\site-packages\optuna\progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-03-04 20:41:49,335] Trial 0 finished with value: 29236.318496308017 and parameters: {'n_estimators': 80, 'max_depth': 20, 'max_features': 11, 'min_impurity_decrease': 4}. Best is trial 0 with value: 29236.318496308017.
[I 2023-03-04 20:41:49,805] Trial 1 finished with value: 29442.97729453431 and parameters: {'n_estimators': 82, 'max_depth': 22, 'max_features': 17, 'min_impurity_decrease': 2}. Best is trial 0 with value: 29236.318496308017.
[I 2023-03-04 20:41:50,325] Trial 2 finished with value: 29166.29792534038 and parameters: {'n_estimators': 97, 'max_depth': 14, 'max_features': 12, 'min_impurity_decrease': 0}. Best is trial 2 with value: 29166.29792534038.
[I 2023-03-04 20:41:50,755] Trial 3 finished with value: 29431.393775417244 and parameters: {'n_estimators': 93, 'max_depth': 11, 'max_features': 10, 'min_impurity_decrease': 0}. Best is trial 2 with value: 29166.29792534038.
[I 2023-03-04 20:41:51,323] Trial 4 finished with value: 29470.319660104666 and parameters: {'n_

In [35]:
optuna.logging.set_verbosity(optuna.logging.ERROR) #关闭自动打印的info，只显示进度条
#optuna.logging.set_verbosity(optuna.logging.INFO)
best_params, best_score = optimizer_optuna(300,"TPE")

  0%|          | 0/300 [00:00<?, ?it/s]


 
 best params:  {'n_estimators': 85, 'max_depth': 10, 'max_features': 16, 'min_impurity_decrease': 1} 
 
 best score:  [28790.39833062145] 



In [36]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
best_params, best_score = optimizer_optuna(300,"GP")

  0%|          | 0/300 [00:00<?, ?it/s]


 
 best params:  {'n_estimators': 100, 'max_depth': 10, 'max_features': 16, 'min_impurity_decrease': 0} 
 
 best score:  [28692.908874419536] 

